# RBM: Restricted Boltzmann Machine with MNIST dataset

- In this tutorial, we will utilize MxNet NDArray API to compose RBM(Restricted Boltzmann Machine) model, in initializing parameters before starting real neural network.
- This tutorial is for beginners who might want to know weight initialization and brief usage of basic MxNet API.

## Prerequisites


You need to install 
    - Python 2.7. https://www.python.org/downloads/
    - mxnet(gpu ver.) https://mxnet.incubator.apache.org/get_started/install.html


This is to use RBM model on gluon Project. RBM is Introduced by (Hinton), this model is to initialize weight efficiently. This is not to predict output, but to obtain the distribution of input data. If you want to know more, please refer to the article.

- A Practical Guide to Training : Restricted Boltzmann Machines(Geoffrey Hinton)


## The Data

 MNIST data, well known for basic step for neural network, is what we deal with. You can easily download data via MxNet or from LeCun's website.
     - http://yann.lecun.com/exdb/mnist/
     
     
## Prepare the Data

#### 1. For training to initialize weight, we need to load data. It is easy process.
 



In [5]:
# -*- coding: utf-8 -*-

import cPickle
import gzip
import os

import numpy

def load_data(dataset='mnist.pkl.gz'):
  dataset = os.path.join(os.path.split("__file__")[0], 'data', dataset)
  f = gzip.open(dataset, 'rb')
  train_set, valid_set, test_set = cPickle.load(f)
  f.close()

  def make_numpy_array(data_xy):
    data_x, data_y = data_xy
    return numpy.array(data_x), numpy.array(data_y)

  train_set_x, train_set_y = make_numpy_array(train_set)
  valid_set_x, valid_set_y = make_numpy_array(valid_set)
  test_set_x, test_set_y = make_numpy_array(test_set)

  rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y),
          (test_set_x, test_set_y)]

  return rval

datasets = load_data('mnist.pkl.gz')
train_set_x, train_set_y = datasets[0]

## Create the Model and initialize Weight
RBM is a model based on Energy Model, which shows quite good performance before real neural network. Weights between visible layer(input) and hidden layer are initialized via prop up and down process.

![Alt text](http://deeplearning.net/tutorial/_images/rbm.png)

In [6]:
import mxnet as mx
class RBM(object):

    def __init__(self, n_hiddens=10,
                 epsilon=0.1,
                 W=None,
                 b=None,
                 c=None,
                 n_samples=784,
                 epochs=2):
        self.n_hiddens = n_hiddens
        self.W = W
        self.b = b
        self.c = c
        self.epsilon = epsilon
        self.n_samples = n_samples
        self.epochs = epochs
        self.h_samples = None

    def sigmoid(self, x):
        """
        Sigmoid logistic function 
        """
        return 1. / (1. + numpy.exp(-numpy.maximum(numpy.minimum(x, 30), -30)))

    def propup(self, vis):
        """
        propagate between layers
        """
        return self.prob_h(vis)

    def prob_h(self, vis):
        """
        calculate the probability with given visible layer
        """
        W_np = self.W.asnumpy()
        return self.sigmoid(numpy.dot(vis, W_np) + self.b)

    def sample_h_given_v(self, vis):
        """
        sampling hidden layer with given visible layer
        """
        return numpy.random.binomial(1, self.propup(vis))

    def propdown(self, hid):
        """
        propagate between layers
        """
        W_np = self.W.asnumpy()
        return self.sigmoid(numpy.dot(hid, W_np.T) + self.c)

    def sample_v_given_h(self, hid):
        """
        sample visible layer with given hidden layer
        """
        return numpy.random.binomial(1, self.propdown(hid))

    def free_energy(self, vis):
        """
        calculate free engergy. Note that the result is negative.
        """
        W_np = self.W.asnumpy()
        return - numpy.dot(vis, self.c) \
               - numpy.log(1. + numpy.exp(numpy.dot(vis, W_np) + self.b)).sum(1)

    def gibbs_chain(self, vis):
        """
        gibbs chaning between layers
        """
        h_ = self.sample_h(vis)
        v_ = self.sample_v(h_)
        return v_

    def _update(self, v_pos, verbose=True):
        """
        update halper
        """
        h_pos = self.propup(v_pos)
        v_neg = self.sample_v_given_h(self.h_samples)
        h_neg = self.propup(v_neg)
        
        W_np = self.W.asnumpy()
        W_np += self.epsilon * (numpy.dot(v_pos.T, h_pos)
                                  - numpy.dot(v_neg.T, h_neg)) / self.n_samples
        self.W = mx.nd.array(W_np)
        self.b += self.epsilon * (h_pos.mean(0) - h_neg.mean(0))
        self.c += self.epsilon * (v_pos.mean(0) - v_neg.mean(0))

        self.h_samples = numpy.random.binomial(1, h_neg)

        return self.pseudo_likelihood(v_pos)

    def pseudo_likelihood(self, v):
        """
        calculate the cost
        """
        fe = self.free_energy(v)

        v_ = v.copy()
        i_ = numpy.random.randint(0, v.shape[1], v.shape[0])
        v_[range(v.shape[0]), i_] = v_[range(v.shape[0]), i_] == 0
        fe_ = self.free_energy(v_)

        return v.shape[1] * numpy.log(self.sigmoid(fe_ - fe))

    def update(self, X, verbose=False):
        """
        update cost
        """
        if self.W == None:
            self.W = numpy.asarray(numpy.random.normal(0, 0.01,
                (X.shape[1], self.n_hiddens)), dtype=X.dtype)
            self.W = mx.nd.array(self.W)
            self.b = numpy.zeros(self.n_hiddens, dtype=X.dtype)
            self.c = numpy.zeros(X.shape[1], dtype=X.dtype)
            self.h_samples = numpy.zeros((self.n_samples, self.n_hiddens),
                                         dtype=X.dtype)

        inds = range(X.shape[0])

        numpy.random.shuffle(inds)

        n_batches = int(numpy.ceil(len(inds) / float(self.n_samples)))

        for epoch in range(self.epochs):
            print(epoch)
            pl = 0.
            for minibatch in range(n_batches):
                pl += self._update(X[inds[minibatch::n_batches]]).sum()
            pl /= X.shape[0]
            print(self.W)
            print ("Epoch %d, Pseudo-Likelihood = %.2f" % (epoch, pl))


    def main():
        pass


    if __name__ == '__main__':
        main()


/home/ubuntu/anaconda3/envs/mxnet_p27/lib/python2.7/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


    - train and get the intialized Weight and dump it.

In [7]:
X = numpy.array(train_set_x)
model = RBM(n_hiddens=10, n_samples=784,epochs=10)
model.update(X)

print(model.W)

cPickle.dump(model.W, open('weight.p', "wb"))


0

[[-0.2669937  -0.26080966 -0.2550104  ..., -0.26267484 -0.27630749
  -0.26593226]
 [-0.26792291 -0.26412126 -0.27305239 ..., -0.28993195 -0.29635659
  -0.27198008]
 [-0.28744251 -0.26321027 -0.26610094 ..., -0.28906548 -0.27673733
  -0.27588615]
 ..., 
 [-0.28515878 -0.28254196 -0.27648616 ..., -0.28323209 -0.25867426
  -0.29814324]
 [-0.28058594 -0.28413859 -0.27928758 ..., -0.27831089 -0.27733001
  -0.28026232]
 [-0.27965176 -0.26214284 -0.28712091 ..., -0.28626716 -0.27738428
  -0.28456607]]
<NDArray 784x10 @cpu(0)>
Epoch 0, Pseudo-Likelihood = -278.24
1

[[-0.31722581 -0.31104183 -0.30520838 ..., -0.31287125 -0.32648882
  -0.31620175]
 [-0.31315628 -0.30939487 -0.31827205 ..., -0.33511931 -0.34155318
  -0.31724405]
 [-0.33147728 -0.30728689 -0.310119   ..., -0.3330701  -0.32075664
  -0.3199743 ]
 ..., 
 [-0.3306635  -0.32808211 -0.32197085 ..., -0.32873008 -0.30414727
  -0.3436805 ]
 [-0.32325622 -0.32684505 -0.32192981 ..., -0.3209655  -0.31998613
  -0.32297391]
 [-0.32545409 -

## Check if RBM-initialized weight works well.
- from Gluon tutorial

In [10]:
from __future__ import print_function
import mxnet as mx
import numpy as np
import pickle as cPickle
from mxnet import nd, autograd, gluon
from mxnet.gluon import nn, Block
mx.random.seed(1)

###########################
#  Speficy the context we'll be using
###########################
ctx = mx.gpu(0)

###########################
#  Load up our dataset
###########################
batch_size = 64
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)
def relu(X):
    return nd.maximum(X, 0)
class MyDense(Block):
    ####################
    # We add arguments to our constructor (__init__)
    # to indicate the number of input units (``in_units``) 
    # and output units (``units``)
    ####################
    def __init__(self, units, in_units=0, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        with self.name_scope():
            self.units = units
            self._in_units = in_units
            #################
            # We add the required parameters to the ``Block``'s ParameterDict , 
            # indicating the desired shape
            #################
            self.weight = self.params.get('weight', init=mx.init.Uniform(scale=1), shape=(in_units, units))            
            self.bias = self.params.get('bias', shape=(units,))
            print(self.weight)
    #################
    #  Now we just have to write the forward pass. 
    #  We could rely upong the FullyConnected primitative in NDArray, 
    #  but it's better to get our hands dirty and write it out
    #  so you'll know how to compose arbitrary functions
    #################
    def forward(self, x):
        with x.context:
            linear = nd.dot(x, self.weight.data()) + self.bias.data()
            activation = relu(linear)
            return activation
        
dense = MyDense(10, in_units= 28*28)
dense.collect_params().initialize(ctx=ctx)
dense.weight.set_data(cPickle.load(open('weight.p')))
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(dense)
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})
metric = mx.metric.Accuracy()

def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    
    for i, (data, label) in enumerate(data_iterator):
        with autograd.record():
            data = data.as_in_context(ctx).reshape((-1,784))
            label = label.as_in_context(ctx)
            label_one_hot = nd.one_hot(label, 10)
            output = net(data)
        
        metric.update([label], [output])
    return metric.get()[1]
epochs = 5  # Low number for testing, set higher when you run!
moving_loss = 0.

for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx).reshape((-1,784))
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
            cross_entropy.backward()
        trainer.step(data.shape[0])
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Train_acc %s, Test_acc %s" % (e, train_accuracy, test_accuracy))


Parameter mydense1_weight (shape=(784, 10), dtype=<type 'numpy.float32'>)
Epoch 0. Train_acc 0.903085714286, Test_acc 0.9093
Epoch 1. Train_acc 0.907714285714, Test_acc 0.9045125
Epoch 2. Train_acc 0.910352380952, Test_acc 0.908293333333
Epoch 3. Train_acc 0.9122, Test_acc 0.910790909091
Epoch 4. Train_acc 0.913762857143, Test_acc 0.912527586207


## Summary

So far, RBM initialization is quite better than initial learning. WIth Uniform Random Initialization, the accuracy in the first few epoch is very low. However, even if this initialization might be worse than Xavier initialization, it shows quite good initial accuracy